In [ ]:
import pylast
import math
import random
from collections import defaultdict
import time
from IPython.display import clear_output
from pylast import WSError

API_KEY = "7008b5589d58885ace36d3221e86cbd0"
API_SECRET = "a3f47c877288ee66e42a15bc603cffb6"
network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

class Artist:
    #tags -> top three tags
    def __init__(self, ID, name, image, tags = None):
        self.id = ID
        self.name = name
        self.image = image
        self.tags = tags

class Song:

    #image -> cover image
    #tags -> top three tags
    def __init__(self, ID, artist, name, image, tags = None, playcount = None):
        self.id = ID
        self.artist = artist
        self.name = name
        self.image = image
        self.tags = tags
        self.playCount = playcount

def TryGetCover(track: pylast.Track):
    try:
        return track.get_cover_image()
    except Exception:
        return None
def TryGetImage(artist: pylast.Artist):
    try:
        return artist.get_image()
    except Exception:
        return None
def TryGetTags(track: pylast.Track):
    try:
        return track.get_top_tags()
    except Exception:
        return None

def initialQuery(type):

    query = input(f"Enter search query for favorite {type}: ")

    itemIDs = []
    itemText = []
    itemObjects = []
    count = 0

    #Queries database, then displays
    if type == "track":
        searchResults = network.search_for_track("", query).get_next_page()
        for item in searchResults[:10]:
            artistName = item.get_artist().get_name()
            trackName = item.get_title()
            itemText.append(f"[{count}]: {trackName} by {artistName}")
            itemIDs.append(f"{artistName} - {trackName}")
            itemObjects.append(item)
            count += 1

    elif type == "artist":
        searchResults = network.search_for_artist(query).get_next_page()
        for item in searchResults[:10]:
            artistName = item.get_name()
            itemText.append(f"[{count}]: {artistName}")
            itemIDs.append(artistName)
            itemObjects.append(item)
            count += 1
    else: 
        return "Invalid query"

    print("-------------RESULTS-------------")
    for entry in itemText:
        print(entry)

    while True:
        try:
            index = int(input(f"Pick a {type} by index (0 to {count - 1}): "))
            if 0 <= index < count:
                break
            print("Invalid index.")
        except ValueError:
            print("Enter a number.")

    selected = itemObjects[index]

    if type == "track":
        track = network.get_track(selected.get_artist().get_name(), selected.get_title())
        title = track.get_title()
        artistId = track.get_artist().get_mbid()
        artistName = track.get_artist().get_name()
        return (track, Song(
                ID = f"{artistName} - {title}",
                artist = artistName,
                name = title,
                image = TryGetCover(track),
                tags = None
            ))
           

    elif type == "artist":
        artist = network.get_artist(selected.get_name())
        id = artist.get_mbid()
        return (artist, Artist(
            ID = id,
            name = artist.get_name(),
            image = TryGetImage(artist), 
            tags = None
        ))






#trackSeed,artistSeed are pylast objects
#I know that just calling get_recommended on like 400 tracks is cheating, so I want to create a track pool with varying sources
#1/4 of the tracks will be generated from lastFM's recommendation algorithm (simple: quarter cheating)
#1/4 of the tracks will be generated from related artists
#Remaining tracks generated from the highest frequency tags of the tracks form the last three
#get some random artists -> get random albums -> get random songs from each

#After, pad with some random songs

#outputs custom objects not pylast objects
def generateTrackPool(count: int, trackSeed: pylast.Track, artistSeed: pylast.Artist = None):
    
    #I found that there's some edge cases that arise when you don't generate enough tracks, so impose some minimum
    if(count < 20):
        raise ValueError("Count needs to be >= 20")
        return 0
    #Throttles execution to not be caught by rate limitter
    timeRateLimit = count/10

    #generate seed integers to get varying track amounts per step 
    x = 0
    y = 0
    z = 0

    while(True):
        #SYSTEM PARAMETERS
        sigma = count * (1/8)
        mean = count * 1/4
        x = math.floor(random.gauss(mean, sigma))
        y = math.floor(random.gauss(mean, sigma))
        z = count - x - y
        if(x > 0 and y > 0 and z > 0):
            break
            
    pool = set() #Filled with Song class from this code
    uniqueTags = defaultdict(int)
    uniqueArtists = set() #Filled with Artist class from this code

    #Adding seed artist
    uniqueArtists.add(Artist(ID = artistSeed.get_mbid() , name = artistSeed.get_name() , image = TryGetImage(artistSeed), tags = None))

    #Bias the tags to the seed track -> Throughout this code, I use that lastFM api returns the tags from most frequent to least
    seedTags = trackSeed.get_top_tags()[:3]

    for tt in seedTags:
        #Picked count/5 arbitrarily, but I think this should make sure step 3 uses the best tags
        tag = tt.item
        uniqueTags[tag.get_name()] += math.floor(count/5)



    #Step one: generate x similar tracks TO seed track doing BFS-kinda thing
    tempTrack = trackSeed
    done = False

    while not done:
        try:
            similarTracks = tempTrack.get_similar(limit = x)
        except WSError:
            print("Similar tracks not found, shutting down.")
            return
        update = True
        for similar in similarTracks:
            
            track = similar.item #pylist.track object

            if update:
                tempTrack = track
                update = False

            tags = TryGetTags(track)

            #Edge case where song doens't have tags
            if(tags == None):
                 pool.add(Song(
                    ID = track.get_mbid(),
                    artist = track.get_artist().get_name(),
                    name = track.get_name(),
                    image = TryGetImage(track),
                    tags = None,
                    playcount= track.get_playcount()
                ))
            else:

                firstTags = [l.item for l in tags[:3]]
                for u in firstTags:
                    uniqueTags[u.get_name()] += 1

                pool.add(Song(
                    ID = track.get_mbid(),
                    artist = track.get_artist().get_name(),
                    name = track.get_name(),
                    image = TryGetImage(track),
                    tags = firstTags,
                    playcount= track.get_playcount()
                ))
            if(len(pool) == x - 1):
                done = True
                break
    #throttle for rate limits
    print("--->Step 1 done<---")
    time.sleep(timeRateLimit)   
    
    #Step two: generate y similar tracks to seed album, BFS kind of thing again
    done = False
    tempArtist = artistSeed
    tempCount = 0

    while not done:
        similarArtists = tempArtist.get_similar(limit=3)
        update = True
        for similar in similarArtists:
            artist = similar.item  #pylast.Artist object

            if update:
                tempArtist = artist
                update = False

            uniqueArtists.add(Artist(
                ID=artist.get_mbid(),
                name=artist.get_name(),
                image=TryGetImage(artist),
                tags=None
            ))
            
            albums = artist.get_top_albums(limit=2)


            for top in albums:
                album = top.item

                #Accounting for edge case of track being a single
                try:
                    tracks = album.get_tracks()
                except pylast.WSError:
                    continue

                randomTracks = [t for t in tracks if random.choice([True, False])] #<- from stack exchange
                for track in randomTracks:
                    #Theres some tracks that have bad database parameters for some reason, so wrap in try catch (bad I know)
                    try:
                        #Some errors from get_top_tags being blank, idk why pylast works like this but it does whatever
                        tags = TryGetTags(track)
                        if(tags == None):
                            pool.add(Song(
                                ID=track.get_mbid(),
                                artist=track.get_artist().get_name(),
                                name=track.get_name(),
                                image=TryGetImage(track),
                                tags=None,
                                playcount=track.get_playcount()
                            ))
                        else:
                            firstTags = [l.item for l in tags[:3]]
                            for u in firstTags:
                                uniqueTags[u.get_name()] += 1

                            pool.add(Song(
                                ID=track.get_mbid(),
                                artist=track.get_artist().get_name(),
                                name=track.get_name(),
                                image=TryGetImage(track),
                                tags=firstTags,
                                playcount=track.get_playcount()
                            ))

                        tempCount += 1
                        if tempCount == y:
                            done = True
                            break
                    except WSError:
                        continue
                if done:
                    break
            if done:
                break
    print("--->Step 2 done<---")            
    time.sleep(timeRateLimit)  
    #Step three -> Run through tags generate floor(z/3) for each of the top tags
    j = math.floor(z/3)
    for i in range(0,3):
        #Take the max frequency tag, then find songs as usual, then remove it from the dictionary when done
        maxTagName = max(uniqueTags, key = uniqueTags.get) #<- from stack exchange

        #Loop count values, count tracks how many songs have been added
        #previousLength tracks the past length of pool to make sure the values are added successfully
        tempCount = 0
        previousLength = len(pool)

        tag = network.get_tag(maxTagName)
        similarTracks = tag.get_top_tracks(limit = j + 10)
        for similar in similarTracks:
            track = similar.item #pylist.track object
            #Don't add tags back to uniqueTags (not the point here)
            tags = track.get_top_tags()
            pool.add(Song(
                ID = track.get_mbid(),
                artist = track.get_artist().get_name(),
                name = track.get_name(),
                image = TryGetImage(track),
                tags = [l.item.get_name() for l in tags[:3]],
                playcount=track.get_playcount()
            ))
            #Value added successfully
            if(len(pool) == previousLength + tempCount + 1):
                tempCount += 1
            if(tempCount == j):
                break   
            
        #Then remove top frequency item
        del uniqueTags[maxTagName]
        time.sleep(timeRateLimit/3)

            
    print("--->Step 3 done<---")
    return pool


(t, tempObjectT) = initialQuery("track")
(a, tempObjectA) = initialQuery("artist")
clear_output(wait= True)
print(f"Seed track: {tempObjectT.name}")
print(f"Seed artist: {tempObjectA.name}")


trackPool = generateTrackPool(40, t, a)
#TRACK POOL HALFWAY DONE BUT USABLE, tracks use song object
#WARNING: using seed track/artist as lesser known causes the runtime to drastically increase

print(tempObjectT.name)
print(len(trackPool))
for  y in trackPool:
    print(f"{y.name} -- {y.artist},       {y.playCount} plays")

Seed track: Cruel Summer
Seed artist: Taylor Swift
--->Step 1 done<---
--->Step 2 done<---
--->Step 3 done<---
Cruel Summer
39
Felt Good About You -- Gracie Abrams,    playcount: 2358408
Midnight City -- M83,    playcount: 20103742
LoveGame -- Lady Gaga,    playcount: 16494215
Midnight City -- M83,    playcount: 20103742
Poker Face -- Lady Gaga,    playcount: 28221781
Yellow -- Coldplay,    playcount: 29464111
Close to you -- Gracie Abrams,    playcount: 9370153
Viva la Vida -- Coldplay,    playcount: 30506640
I Can Do It With a Broken Heart -- Taylor Swift,    playcount: 15079055
The Less I Know the Better -- Tame Impala,    playcount: 35380892
enough for you -- Olivia Rodrigo,    playcount: 11657366
Walking on a Dream -- Empire of the Sun,    playcount: 16541178
Poker Face -- Lady Gaga,    playcount: 28221781
hope ur ok -- Olivia Rodrigo,    playcount: 9340526
Billie Jean -- Michael Jackson,    playcount: 18709354
All The Stars (with SZA) -- Kendrick Lamar,    playcount: 23404626
Die